# 🚶 Simulación de Trayectos con Agentes

Este cuaderno utiliza el framework `mesa` para simular trayectos de trabajadores desde sus residencias a sus lugares de trabajo. Cada agente representa una persona que se desplaza diariamente.

In [ ]:
# Instalación requerida si no tienes mesa
# !pip install mesa

In [ ]:
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
import pandas as pd
import random

## 📄 Cargar datos de trayectos

In [ ]:
df = pd.read_csv('data/processed/datos_movilidad_coordenadas.csv')
df = df.dropna(subset=['Lat_Residencia', 'Lon_Residencia', 'Lat_Trabajo', 'Lon_Trabajo'])
df.reset_index(drop=True, inplace=True)
df.head()

## 👤 Definición de los agentes y el modelo

In [ ]:
class Trabajador(Agent):
    def __init__(self, unique_id, model, origen, destino):
        super().__init__(unique_id, model)
        self.origen = origen
        self.destino = destino
        self.posicion_actual = origen
        self.trayectoria = [origen]
        self.terminado = False

    def step(self):
        if self.terminado:
            return
        lat, lon = self.posicion_actual
        dlat = (self.destino[0] - lat) * 0.1
        dlon = (self.destino[1] - lon) * 0.1
        nueva_pos = (lat + dlat, lon + dlon)
        self.posicion_actual = nueva_pos
        self.trayectoria.append(nueva_pos)

        if abs(dlat) < 0.0001 and abs(dlon) < 0.0001:
            self.terminado = True

In [ ]:
class SimuladorDesplazamiento(Model):
    def __init__(self, df):
        self.schedule = SimultaneousActivation(self)
        self.trabajadores = []

        for i, row in df.iterrows():
            origen = (row['Lat_Residencia'], row['Lon_Residencia'])
            destino = (row['Lat_Trabajo'], row['Lon_Trabajo'])
            agente = Trabajador(i, self, origen, destino)
            self.schedule.add(agente)
            self.trabajadores.append(agente)

    def step(self):
        self.schedule.step()

## ▶️ Ejecutar la simulación

In [ ]:
modelo = SimuladorDesplazamiento(df)

for _ in range(20):
    modelo.step()

## 📈 Visualización simple de las trayectorias

In [ ]:
import folium
m = folium.Map(location=[df['Lat_Residencia'].mean(), df['Lon_Residencia'].mean()], zoom_start=10)

for agente in modelo.trabajadores:
    folium.PolyLine(agente.trayectoria, color='blue', weight=2, opacity=0.6).add_to(m)

m